In [ ]:
!pip install polars
!pip install fugashi[unidic]

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# こちらの共有フォルダにアクセスしてください
# https://drive.google.com/drive/folders/1EyZuU3SRCnQ57VkpTR3_a7sOFVEl6Uz5

prefix_files = '/content/drive/Shareddrives/ai_lab_2023'

# 演習
## LDAモデルを使って、類似文書の検索
1. livedoor newsコーパスのカテゴリーごとに８：２の割合でデータセットを学習用・テスト用に分割してください
1. トピック数を2~10の中で、最もパープレキシティの低いLDAモデルを作ってください
1. 各カテゴリーごとのテストデータを入力として使って、LDAモデルによって最も類似度の高い文書を学習データから選定してください。
1. 上記で、選定されたデータが入力したデータと同じカテゴリーかどうかを判定してください。同じカテゴリーの場合は成功とします。
1. カテゴリーごとに成功率を計算してください。

## word2vec/k-meansとLDAの比較
1. 上記で学習させたLDAの各トピックの上位10個ずつ単語を抽出してください。
1. これらの単語をword2vecで単語ベクトルに変換してください。
1. これらの単語ベクトル集合をk-meansでクラスタリングしてください。ただし、k-meansのクラスタ数はLDAのトピック数と同じにしてください。
1. 1で抽出したLDAのトピックの単語集合とk-meansのクラスタの単語集合を比較してください。

k-meansはこちら：https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
import re
import requests
import polars as pl
from fugashi import Tagger
from gensim.corpora import Dictionary, MmCorpus
from gensim import models
from sklearn.model_selection import train_test_split

class LivedoorCorpus():
    def __init__(self, df):
        self.df = df

        # 全角半角文字以外（記号と数字）を正規表現を使って除去
        pattern = r"[^\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\u20000-\u2ffff\sa-zA-Z]"
        self.raw_documents = [re.sub(pattern, "", text) for text in self.df["DOCUMENT"]]
        # Mecabで分かち書きして、単語に分割
        self.raw_documents = [Tagger('-Owakati').parse(text).split() for text in self.raw_documents]
        # ストップワードの除去
        self.raw_documents = self._rm_stopwords()
        # 1文字は除去
        self.raw_documents = [[word for word in text if len(word) > 1]for text in self.raw_documents]

        self.dictionary = Dictionary(self.raw_documents)

        self.bow = [ self.dictionary.doc2bow(text) for text in self.raw_documents]


    def reset_dict_corpus(self):
        self.dictionary = Dictionary(self.raw_documents)
        self.bow = [ self.dictionary.doc2bow(text) for text in self.raw_documents]

    def print_stats(self):
        print(f"文書数: {self.dictionary.num_docs}, " + f"語彙数: {len(self.dictionary)}")

    def dict_top_n(self, top_n: int):
        most_frequent_ids = (v for v in self.dictionary)
        most_frequent_ids = sorted(most_frequent_ids, key=self.dictionary.dfs.get, reverse=True)
        most_frequent_ids = most_frequent_ids[:top_n]
        return [self.dictionary[idx] for idx in most_frequent_ids]
        
    def _rm_stopwords(self):
        # ストップワードの準備
        stopwords_url = "http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt"
        r = requests.get(stopwords_url)
        tmp = r.text.split('\r\n')
        stopwords = []
        for i in range(len(tmp)):
            if len(tmp[i]) < 1:
                continue
            stopwords.append(tmp[i])

        return [[word for word in text if not word in stopwords]for text in self.raw_documents]

In [ ]:
train_corpus.reset_dict_corpus()
train_corpus.print_stats()
train_corpus.dictionary.filter_extremes(10,0.5)
train_corpus.print_stats()
train_bow = [ train_corpus.dictionary.doc2bow(text) for text in train_corpus.raw_documents]
# 辞書はtrain_corpusのものを使います。
test_bow = [ train_corpus.dictionary.doc2bow(text) for text in test_corpus.raw_documents]

In [ ]:
import pickle

with open(f'{prefix_files}/train_corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)
dictionary = Dictionary.load(f'{prefix_files}/livedoor.dict')
doc_train = MmCorpus(f'{prefix_files}/train_bow.mm')
doc_test = MmCorpus(f'{prefix_files}/test_bow.mm')

In [ ]:
topic_range = ...

def calc_perplexity(m, c):
    import numpy as np
    return None

def search_model(corpus_train, corpus_test):
    most = [1.0e6, None]
    print(f"dataset: training/test = {len(corpus_train)}/{len(corpus_test)}")

    for t in topic_range:
        # 辞書はtrain_corpusのものを使います。
        m = ...
        p1 = calc_perplexity(m, corpus_train)
        p2 = calc_perplexity(m, corpus_test)
        print(f"{t}: perplexity is {p1}/{p2}")
        
        if p2 < most[0]:
            most[0] = p2
            most[1] = m
    
    return most[0], most[1]

perplexity, model_lda = search_model(train_bow, test_bow)
print(f"Best model: topics={model_lda.num_topics}, perplexity={perplexity}")

In [ ]:
import numpy as np
from gensim import similarities
index = ...

acc = []
loss = []
for i, doc in enumerate(test_bow):
    similarity_lda = ...
    raise NotImplementedError("判定する処理を書いてください。")


In [ ]:
len(acc)/(len(acc) + len(loss))

In [ ]:
class WVCorpus():
    def __init__(self, corpus):
        self.corpus = corpus
    def __iter__(self):
        return iter(self.corpus)

sentences = WVCorpus(train_corpus.raw_documents)
# instantiating and training the Word2Vec model
model_wv = ...

# getting the training loss value
training_loss = model_wv.get_latest_training_loss()
print(training_loss)

In [ ]:
from sklearn.cluster import KMeans

topic_words = []

raise NotImplementedError("単語の抽出、単語ベクトルへの変換、クラスタリング、ラベル合わせ")

In [ ]:
topic_df

In [ ]:
for k,v in cluster_words.items():
    print(k,v)